In [1]:
from dotenv import load_dotenv
load_dotenv()

from app import api_client, mailgun_util
from pathlib import Path
import os
import pandas as pd
pd.set_option("display.max_columns", None)
contacts = api_client.filter_contacts({})
CONTACTS_CSV_PATH = Path(os.environ["CONTACTS_CSV_PATH"])

df = pd.DataFrame(contacts)

In [2]:
df

,id,contact_name,company_name,email,first_name,last_name,contact_type,created_time,outreach_idx,auto_number,last_touch_date,stage,notes,unsub,intro_date,dfu1_date,dfu2_date
0,zcrm_886415000000509378,Daniel Muhlbach,eCom Teams Sweden AB,daniel.muhlbach@footway.com,Daniel,Muhlbach,Personal,2025-09-04 14:08:17,2,1,,intro,,false,2025-09-23_11:02:11,2025-09-30_11:02:11,
1,zcrm_886415000000509379,Hercules Ostberg,Bodyflight Sweden AB,hercules.ostberg@bodyflight.se,Hercules,Ostberg,Personal,2025-09-04 14:08:17,2,2,,intro,,false,2025-09-23_11:02:11,2025-09-30_11:02:11,
2,zcrm_886415000000509380,Anders Bjorkenbo,Sustainable Energy Solutions Sweden Holding AB...,anders.bjorkenbo@sens.se,Anders,Bjorkenbo,Personal,2025-09-04 14:08:17,2,3,,intro,,false,2025-09-23_11:02:11,2025-09-30_11:02:11,
3,zcrm_886415000000509381,Carl-Johan Ivarsson,Qlucore AB,carl-johan.ivarsson@qlucore.com,Carl-Johan,Ivarsson,Personal,2025-09-04 14:08:17,2,4,,intro,,false,2025-09-23_11:02:11,2025-09-30_11:02:11,
4,zcrm_886415000000509382,Lena Sundquist,Climeon AB,lena.sundquist@climeon.com,Lena,Sundquist,Personal,2025-09-04 14:08:17,2,5,,intro,,false,2025-09-23_11:02:11,2025-09-30_11:02:11,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,zcrm_886415000000511684,Hello Flowlife AB,Flowlife AB,hello@flowlife.com,Hello,Flowlife AB,Press/Generic,2025-09-04 14:30:24,,194,,new,,false,,,
194,zcrm_886415000000511685,Info ProteinBolaget i Sverige AB,ProteinBolaget i Sverige AB,info@proteinbolaget.se,Info,ProteinBolaget i Sverige AB,Press/Generic,2025-09-04 14:30:24,,195,,new,,false,,,
195,zcrm_886415000000511686,Info Maurten AB,Maurten AB,info@maurten.com,Info,Maurten AB,Press/Generic,2025-09-04 14:30:24,,196,,intro,,false,,2025-09-24_11:02:11,
196,9e5f8b311471427f9a698843e7f5bd88,,VDS,david.vikstrand@gmail.com,David,Vikstrand,,2025-09-23 11:03:22,,197,,new,,,,,


In [36]:
df.loc[(df["outreach_idx"] == "") & (df["intro_date"] != ""), "dfu1_date"] = ""

In [14]:
import requests
from datetime import datetime, timezone
from app.mailgun_util import MAILGUN_API_KEY, MAILGUN_DOMAIN

def mailgun_sent_today_count() -> int:
    base = "https://api.eu.mailgun.net" 
    url  = f"{base}/v3/{MAILGUN_DOMAIN}/events"

    # Midnight UTC (Mailgun’s limits are best treated as UTC calendar days)
    begin = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
    begin_s = begin.strftime("%a, %d %b %Y %H:%M:%S %Z")  # e.g., "Tue, 23 Sep 2025 00:00:00 UTC"

    params = {"event": "accepted", "begin": begin_s, "ascending": "yes", "limit": 300}
    total = 0
    auth  = ("api", MAILGUN_API_KEY)

    while True:
        r = requests.get(url, auth=auth, params=params, timeout=30)
        r.raise_for_status()
        j = r.json()
        items = j.get("items", [])
        total += len(items)

        # follow pagination
        next_url = (j.get("paging") or {}).get("next")
        if not items or not next_url:
            break
        url, params = next_url, None  # next URL already includes query

    return total

cnt = mailgun_sent_today_count()

In [15]:
cnt

42

In [2]:
from app.tasks import mailgun_send_campaign
mailgun_send_campaign.main(["intro", "david.vikstrand@gmail.com"])

[info] using Mailgun tag 'intro_2025-09-23_13:23:57'
[mailgun] stage 'intro' sent template 'intro_v1' to david.vikstrand@gmail.com


0

In [4]:
msg = """
[error] failed to send to josef.hederstrom@lumenradio.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to maria@abas.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to tomas@abas.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to patricia@mapiful.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to edvin@mapiful.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to danny@glomo.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to ben@glomo.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to jacob@fantasmagames.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to fredrik@fantasmagames.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to anton@klimato.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to cecilia@trifilon.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to edh@bioextrax.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to johan@careofgerd.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to maria@dermanord.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to lars@versoskincare.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to newretailers@estellethild.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to babba@ceremonia.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to helena@underyourskin.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to johan@betteryou.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to letters@storaskuggan.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to niclas@agonistparfums.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to ecom@idunminerals.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to mette@mettepicaut.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to eric@mettepicaut.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to marie@mettepicaut.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to jessica@mettepicaut.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to angelica@mettepicaut.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to rs@rosenserien.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to alexander@stutterheim.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to calle@kavat.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to monica@gymleco.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to lucas@flowlife.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to akeson@proteinbolaget.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to bengt@fairfordholdings.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to emilie@fairfordholdings.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to eva@fairfordholdings.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to marie@fairfordholdings.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to mattias@fairfordholdings.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to peter@fairfordholdings.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to robert@fairfordholdings.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to marcus@c3c.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to maria@c3c.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to mwg@ecoride.com: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages
[error] failed to send to jens@textalk.se: 420 Client Error: status code 420 for url: https://api.eu.mailgun.net/v3/for.vdsai.se/messages"""

In [5]:
emails_list_v2 = []
for m in msg.split("\n"):
    if not m.startswith("[error]"): continue
    m = m.split(": 4")[0].split("send to ")[1]
    emails_list_v2.append(m)
    
emails_list_v2
    

['josef.hederstrom@lumenradio.com',
 'maria@abas.se',
 'tomas@abas.se',
 'patricia@mapiful.com',
 'edvin@mapiful.com',
 'danny@glomo.com',
 'ben@glomo.com',
 'jacob@fantasmagames.com',
 'fredrik@fantasmagames.com',
 'anton@klimato.com',
 'cecilia@trifilon.com',
 'edh@bioextrax.com',
 'johan@careofgerd.se',
 'maria@dermanord.se',
 'lars@versoskincare.com',
 'newretailers@estellethild.se',
 'babba@ceremonia.com',
 'helena@underyourskin.se',
 'johan@betteryou.se',
 'letters@storaskuggan.com',
 'niclas@agonistparfums.com',
 'ecom@idunminerals.com',
 'mette@mettepicaut.se',
 'eric@mettepicaut.se',
 'marie@mettepicaut.se',
 'jessica@mettepicaut.se',
 'angelica@mettepicaut.se',
 'rs@rosenserien.com',
 'alexander@stutterheim.se',
 'calle@kavat.com',
 'monica@gymleco.se',
 'lucas@flowlife.com',
 'akeson@proteinbolaget.se',
 'bengt@fairfordholdings.com',
 'emilie@fairfordholdings.com',
 'eva@fairfordholdings.com',
 'marie@fairfordholdings.com',
 'mattias@fairfordholdings.com',
 'peter@fairfordho

In [12]:
# save df
df.to_csv(CONTACTS_CSV_PATH, index=False)

In [13]:
df.loc[df["email"].isin(emails_list_v2)]

,id,contact_name,company_name,email,first_name,last_name,contact_type,created_time,outreach_idx,auto_number,last_touch_date,stage,notes,unsub,intro_date,dfu1_date,dfu2_date
30,zcrm_886415000000509408,Josef Hederstrom,LumenRadio AB,josef.hederstrom@lumenradio.com,Josef,Hederstrom,Personal,2025-09-04 14:08:17,,31,,new,,false,,,
32,zcrm_886415000000511523,Maria ABAS Protect AB,ABAS Protect AB,maria@abas.se,Maria,ABAS Protect AB,Personal,2025-09-04 14:30:24,,33,,new,,false,,,
33,zcrm_886415000000511524,Tomas ABAS Protect AB,ABAS Protect AB,tomas@abas.se,Tomas,ABAS Protect AB,Personal,2025-09-04 14:30:24,,34,,new,,false,,,
36,zcrm_886415000000511527,Patricia Mapiful,Mapiful,patricia@mapiful.com,Patricia,Mapiful,Personal,2025-09-04 14:30:24,,37,,new,,false,,,
37,zcrm_886415000000511528,Edvin Mapiful,Mapiful,edvin@mapiful.com,Edvin,Mapiful,Personal,2025-09-04 14:30:24,,38,,new,,false,,,
38,zcrm_886415000000511529,Danny GLOMO,GLOMO,danny@glomo.com,Danny,GLOMO,Personal,2025-09-04 14:30:24,,39,,new,,false,,,
39,zcrm_886415000000511530,Ben GLOMO,GLOMO,ben@glomo.com,Ben,GLOMO,Personal,2025-09-04 14:30:24,,40,,new,,false,,,
40,zcrm_886415000000511531,Jacob Fantasma Games,Fantasma Games,jacob@fantasmagames.com,Jacob,Fantasma Games,Personal,2025-09-04 14:30:24,,41,,new,,false,,,
41,zcrm_886415000000511532,Fredrik Fantasma Games,Fantasma Games,fredrik@fantasmagames.com,Fredrik,Fantasma Games,Personal,2025-09-04 14:30:24,,42,,new,,false,,,
43,zcrm_886415000000511534,Anton Klimato,Klimato,anton@klimato.com,Anton,Klimato,Personal,2025-09-04 14:30:24,,44,,new,,false,,,
